In [22]:
from bs4 import BeautifulSoup, Tag
import pandas as pd
import requests

In [28]:
domain = 'https://studyinthestates.dhs.gov'
rootPage = requests.get('https://studyinthestates.dhs.gov/sevis-by-the-numbers/sevis-by-the-numbers-data')
rootPage.raise_for_status()

soup = BeautifulSoup(rootPage.content)

In [ ]:
listItems = soup.findAll('div', {'class': 'field-item even'})[0].ul
reports = []
for li in listItems:
    if not isinstance(li, Tag):
        continue
    # capturing first two words of title gets us "Month Year Type"
    title = ' '.join(li.a.string.split(' ')[:3])
    path = li.a['href']
    r = requests.get(domain + path)
    r.raise_for_status()
    reports.append({
        'title': title,
        'dataframe': pd.read_html(r.content)[0]
    })
    print('Done reading {} report.'.format(title))

Done reading April 2014 SEVIS report.
Done reading July 2014 SEVIS report.
Done reading October 2014 SEVIS report.
